In [1]:
from Bio import SeqIO, motifs
from Bio.Seq import Seq
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from collections import Counter
import shutil
import os
import csv
import requests
from bs4 import BeautifulSoup
from io import StringIO
import random
import re

In [2]:
def uppercase_pos(input_string):
    return [index for index, char in enumerate(input_string) if char.isupper()]

In [3]:
def shuffle_sequence(seq):
    seq_list = list(seq)
    random.shuffle(seq_list)
    return ''.join(seq_list)

In [5]:
DNA_inf = pd.read_csv("success_download.csv")
matrix_ID_list = []

for i in range(len(DNA_inf)):
    matrix_ID_list.append(DNA_inf.loc[i,"ID"])
    
matrix_ID_list.remove("MA0036.1")

#### Before that, I only keep the uppercase of the sequence and remove the lowercase for every DNA sequences.
#### To generate the negative data, I random shuffle all the DNA sequence.


In [ ]:
for matrix_name in matrix_ID_list:
    folder = "3_FASTA_80hom/"
    fasta_matrix = f"{folder}{matrix_name}_80hom.fasta"
    with open(fasta_matrix, "r") as file:
        sequences = list(SeqIO.parse(file, "fasta"))
        
    unique_shuffled_sequences = set()
    for sub_sequence in sequences:
        
        if "(" not in sub_sequence.id and ")" not in sub_sequence.id:
            sub_sequence.description = re.sub(r'\([^)]*\)', '', sub_sequence.description)
        else:
            new_string = re.sub(r'\([^)]*\)', '', sub_sequence.description)
            sub_sequence.id = new_string
            sub_sequence.description = new_string
            sub_sequence.name = new_string
            
        check = False
        
        upper_pos = uppercase_pos(sub_sequence.seq)
        if upper_pos == []:
            continue
        else:
            start = sub_sequence.seq[0 : 0]
            seq_list = list(sub_sequence.seq[upper_pos[0]:upper_pos[-1]+1])
            random.shuffle(seq_list)
            lent =  ''.join(seq_list)

            unique_shuffled_sequences.add(lent)
            
            sub_sequence.seq = start + lent
            
    
    with open(f"{matrix_name}_80hom_random.fasta", "w") as fileabc:
        SeqIO.write(sequences, fileabc, "fasta")   
    destination_path = f"FASTA_80hom_random/{matrix_name}_80hom_random.fasta"
    if os.path.exists(destination_path):
        os.remove(destination_path)
    shutil.move(f"{matrix_name}_80hom_random.fasta", "FASTA_80hom_random")
        
    print(f"{matrix_name}_80hom_random.fasta")

#### 1.	Based on the similarity: I make an iteration that will check the similarity of each negative sequence with all the positive sequence according to the longest common subsequence. If there is one negative sequence that has over 0.7 similarity with the positive, this negative sequence will be reshuffle and check from the begin. This method takes too much time since the complexity is too high for the iteration (n3), so the negative data generated in this way is not being used (but it has been generated ^_^).

In [8]:
def longest_common_subsequence(str1, str2):

    len_1 = len(str1) + 1
    len_2 = len(str2) + 1

    dp = [[0 for n in range(len_2)] for m in range(len_1)]
    longest = 0
    lcs_str = ''

    for i in range(1, len_1):
        for j in range(1, len_2):
            if str1[i - 1] == str2[j - 1]:
                if i == 1 or j == 1:
                    dp[i][j] = 1
                else:
                    dp[i][j] = dp[i-1][j-1] + 1

                if dp[i][j] > longest:
                    longest = dp[i][j]
                    lcs_str = str1[i - longest: i]
            else:
                dp[i][j] = 0
    similarity = len(lcs_str)/len_2

    return similarity

In [ ]:
for matrix_name in matrix_ID_list:
    folder_pos = "FASTA_80hom/"
    fasta_matrix_pos = f"{folder_pos}{matrix_name}_80hom.fasta"
    with open(fasta_matrix_pos, "r") as file_pos:
        sequences_pos = list(SeqIO.parse(file_pos, "fasta"))
        
    folder_nag = "FASTA_80hom_random/"
    fasta_matrix_nag = f"{folder_nag}{matrix_name}_80hom_random.fasta"
    with open(fasta_matrix_nag, "r") as file_nag:
        sequences_nag = list(SeqIO.parse(file_nag, "fasta"))
    
    i = 0
    for sub_sequence1 in sequences_nag:
        j = 0
        if i % 50 == 0:
            print(f"-------------negative data {i} line-------------")
        sub_sequence_nag = sub_sequence1.seq
        
        for sub_sequence2 in sequences_pos:
            reroll = False
            #print(f"positive data {j} line")
            sub_sequence_pos = ''.join([char for char in sub_sequence2.seq if char.isupper()])
            similarity = longest_common_subsequence(sub_sequence_nag,sub_sequence_pos)
            
            if similarity >= 0.8:
                print(similarity)
                reroll = True
                
            while reroll == True:
                #print("Re-shuffle",i,j)
                start = sub_sequence1.seq[0:0]
                lent = shuffle_sequence(sub_sequence1.seq)
                sub_sequence1.seq = start + lent
                similarity_list = []
                for sub_sequence2_check in sequences_pos[0:j+1]:
                    #print(f"Re-shuffle: positive data check")
                    sub_sequence_pos_check = ''.join([char for char in sub_sequence2_check.seq if char.isupper()])
                    similarity = longest_common_subsequence(sub_sequence1.seq,sub_sequence_pos_check)
                    similarity_list.append(similarity)
                #print(max(similarity_list))
                if max(similarity_list) < 0.8:
                    reroll = False 
            j += 1
        i += 1
    
    
    with open(f"{matrix_name}_random.fasta", "w") as fileabc:
        SeqIO.write(sequences_nag, fileabc, "fasta")   
    destination_path = f"3_FASTA_80hom_2_negative_similarity/{matrix_name}_random.fasta"
    if os.path.exists(destination_path):
        os.remove(destination_path)
    shutil.move(f"{matrix_name}_random.fasta", "3_FASTA_80hom_2_negative_similarity")
        
    print(f"{matrix_name}_random.fasta")

##### 2.	Based on the frequency matrix: the new negative sequence is generated based on the Frequency matrix of each DNA get from JASPAR. The lower score will have the high probability to be assigned to each position. The negative data generated in this way is used in later processing.

In [ ]:
def check_score(seq):
    score = 0
    for q in range(len(seq)):
        score += motif.__dict__['counts'][seq[q]][q]
    length = len(motif.consensus)
    
    score_max = 0
    for i in range(len(motif.consensus)):
        score_max += motif.__dict__['counts'][motif.consensus[i]][i]
    return score, score_max

In [ ]:
DNA_inf = pd.read_csv("success_download.csv")
matrix_ID_list = []

for i in range(len(DNA_inf)):
    matrix_ID_list.append(DNA_inf.loc[i,"ID"])

In [ ]:
for matrix_name in matrix_ID_list:
    fasta_matrix = f"FASTA_80hom/{matrix_name}_80hom.fasta"
    with open(fasta_matrix, "r") as file:
        sequences = list(SeqIO.parse(file, "fasta"))

    jaspar_matrix = f"3_Frequency_matrix/download_JASPAR/{matrix_name}.jaspar"
    with open(jaspar_matrix) as f:
        jaspar_motifs = motifs.parse(f, "jaspar")

    for motif in jaspar_motifs:
        motif.__dict__['counts']
    
    ls = []   
    for n in range(len(sequences)):
        checkpass = False
        #print(sequences[n].seq, check_score(sequences[n].seq))
    
        while checkpass == False:
            sequences_new = ''
            letters = ['A', 'C', 'G', 'T']
            length = len(motif)
            for i in range(length):
                weights = {'A': motif.__dict__['counts']['A'][i]+1, 'C': motif.__dict__['counts']['C'][i]+1,
                           'G': motif.__dict__['counts']['G'][i]+1, 'T': motif.__dict__['counts']['T'][i]+1}
                letters = list(weights.keys())

                total_inverse = 0
                for weight in weights.values():
                    total_inverse += 1/weight
        
                probabilities = []
                for letter in letters:
                    probabilities.append((1/weights[letter]) / total_inverse)
                new_seq = ''.join(random.choices(letters, weights=probabilities, k=1))
                sequences_new += new_seq
        
            score, max_score = check_score(sequences_new)
        
            if sequences_new not in ls and score <= (max_score/4):
                ls.append(sequences_new)
                checkpass = True
            #print(sequences_new,score, max_score)
            #print("--------------")

    
        sequences[n].seq = sequences[n].seq[0 : 0] + sequences_new
        if "(" not in sequences[n].id and ")" not in sequences[n].id:
            sequences[n].description = re.sub(r'\([^)]*\)', '', sequences[n].description)
        else:
            new_string = re.sub(r'\([^)]*\)', '', sequences[n].description)
            sequences[n].id = new_string
            sequences[n].description = new_string
            sequences[n].name = new_string
        

    
    with open(f"{matrix_name}_matrix_similarity.fasta", "w") as fileabc:
        SeqIO.write(sequences, fileabc, "fasta")   
    destination_path = f"3_FASTA_80hom_3_negative_matrix/{matrix_name}_matrix_similarity.fasta"
    if os.path.exists(destination_path):
        os.remove(destination_path)
    shutil.move(f"{matrix_name}_matrix_similarity.fasta", "3_FASTA_80hom_3_negative_matrix")
      
    print(f"{matrix_name}_matrix_similarity.fasta")